In [1]:
!nvidia-smi

Thu Jan 25 10:35:23 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   40C    P0    64W / 300W |  27523MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import scipy as sp
import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import r2_score
import statistics as stat
import statsmodels.distributions as smd
import os

ML = 100000 #nombre d'echantillons aléatoire
t_init = 0
T  = 0.25


In [3]:
def sabr_trajectory(alpha, beta, rho, f0, sigma0, T, dt, n_sim):
    np.random.seed(42)
    n_steps = int(T / dt)
    
    F = np.zeros((n_sim, n_steps + 1))
    sigma = np.zeros((n_sim, n_steps + 1))
    
    F[:, 0] = f0
    sigma[:, 0] = sigma0
    
    dW1 = np.sqrt(dt) * np.random.randn(n_sim, n_steps)
    dW2 = rho * dW1 + np.sqrt(1 - rho**2) * np.sqrt(dt) * np.random.randn(n_sim, n_steps)
    
    for t in range(n_steps):
        F[:, t+1] = F[:, t] + sigma[:, t] * F[:, t]**beta * dW1[:, t]
        sigma[:, t+1] = sigma[:, t] + alpha * sigma[:, t] * dW2[:, t]
    
    return F, sigma

# Function that saves different models during the training phase
def save_network(network, epoch_label, minibatch):
    save_filename = 'net_{}_{}.pth'.format(epoch_label , minibatch)
    save_path = os.path.join('./SavedModels102', save_filename)
    torch.save(network.state_dict(), save_path)
class Generator(torch.nn.Module):
        def __init__(self, input_neurons, hidden_neurons, output_neurons ):
            super(Generator, self).__init__()
            self.hidden= nn.Linear(input_neurons, hidden_neurons)
            self.hiddenM1= nn.Linear(hidden_neurons, hidden_neurons)
            self.hiddenM2= nn.Linear(hidden_neurons, hidden_neurons)
            #self.hiddenM3= nn.Linear(hidden_neurons, hidden_neurons)
            self.Activ =torch.nn.Tanh()
            #self.Activ =torch.nn.ReLU()

            #self.Activ =torch.sin
            self.eps = 1e-20
            
            self.bach1 = nn.BatchNorm1d(input_neurons)
            self.bach2 = nn.BatchNorm1d(hidden_neurons)
            self.bach3 = nn.BatchNorm1d(hidden_neurons)
            self.bach4 = nn.BatchNorm1d(hidden_neurons)
            self.bach5 = nn.BatchNorm1d(hidden_neurons)

            self.out= nn.Linear(hidden_neurons, output_neurons)
        def forward(self, x):
            #x = self.bach1(x)
            x = self.hidden(x)
            x = self.Activ(x)
            #x = self.bach2(x)           
            x = self.hiddenM1(x)
            x = self.Activ(x)
            #x = self.bach3(x)           
            x = self.hiddenM2(x)
            x = self.Activ(x)
            #x = self.bach4(x)           
            #x = self.hiddenM3(x)            
            #x = self.Activ(x)
            #x = self.bach5(x)           
            x = self.out(x)
            
            x[:,:-1] = torch.sigmoid(x[:,:-1])
            #print(x[:,:-1])
            x[:,-1] = torch.atan(x.clone()[:,-1])
            #print(x[:,-1])
            return x


def Wass2Dim(Cc1, Cc2, X,Y, Nm, b):
    Res=0
    for k in range(Nm):
        if k==0:
            n01=0
            n02=0
            
        else:
            n01=n01+int(Cc1[k-1])
            n02=n02+int(Cc2[k-1])
       # if (n02==0):
           # breakpoint()
        n1=int(Cc1[k])
        n2=int(Cc2[k])
        #print(n01+n1)
        #print(n02+n2)
        McX=torch.zeros(1)
        McY=torch.zeros(1)
        VarX=torch.zeros(1)
        VarY=torch.zeros(1)
        epsilon=10**(-16)
        Lambda=100000
        if (n1 != 0) and (n2!= 0):
            if (min(Y[n02:n02+n2,0,0]) <= max(X[n01:n01+n1,0,0])) and (max(Y[n02:n02+n2,0,0]) >= min(X[n01:n01+n1,0,0])):
                if (n1!=1) and (n2!=1):
                    #print('OUI')
                    McX1=(1/n1)*torch.sum(X[n01:n01+n1,0,1])
                    McX2=(1/n1)*torch.sum(X[n01:n01+n1,1,1])

                    #VarX1=torch.var(X[n01:n01+n1,0])
                    #VarX2=torch.var(X[n01:n01+n1,1])

                    McY1=(1/n2)*torch.sum(Y[n02:n02+n2,0,1])
                    McY2=(1/n2)*torch.sum(Y[n02:n02+n2,1,1])

                    #VarY1=torch.var(Y[n02:n02+n2,0])
                    #VarY2=torch.var(Y[n02:n02+n2,1])
                    
                    #breakpoint()
                    CovX=torch.cov(X[n01:n01+n1,:,1].T)
                    CX=torch.linalg.eig(CovX)
                    
                    CEig=torch.view_as_real(CX[0])
                    S=torch.view_as_real(CX[1])
                    Si=torch.zeros(2,2)
                    Si[:,0]=S[:,0][:,0]
                    Si[:,1]=S[:,1][:,0]
                    #breakpoint()
                    #SqCovX=torch.transpose(Si, 0, 1).mm(torch.diag((CEig[:,0])**(1/2))).mm(Si)
                    SqCovX=Si.mm(torch.diag((CEig[:,0])**(1/2))).mm(torch.transpose(Si, 0, 1))
                    CovY=torch.cov(Y[n02:n02+n2,:,1].T)
                    
                    S14=(SqCovX.mm(CovY)).mm(SqCovX)
                    C2X=torch.linalg.eig(S14)
                    C2Eig=torch.view_as_real(C2X[0])
                    S2=torch.view_as_real(C2X[1])
                    S2i=torch.zeros(2,2)
                    S2i[:,0]=S2[:,0][:,0]
                    S2i[:,1]=S2[:,1][:,0]
                    #breakpoint()
                    #SqCovX=torch.transpose(Si, 0, 1).mm(torch.diag((CEig[:,0])**(1/2))).mm(Si)
                    Sq4=S2i.mm(torch.diag((C2Eig[:,0])**(1/2))).mm(torch.transpose(S2i, 0, 1))


                    #breakpoint()
                    if k==0:
                        if (CovX[0,0] > epsilon) and (CovY[0,0] > epsilon):
                            #B=torch.trace(CovX)+torch.trace(CovY)-2*torch.sqrt(torch.trace((SqCovX.mm(CovY)).mm(SqCovX)))
                            B=torch.trace(CovX)+torch.trace(CovY)-2*torch.trace(Sq4)

                            Res=(McX1-McY1)**2+(McX2-McY2)**2+B
                        else :
                            print('Yes1')
                           #Res=(McX-McY)**2+VarX+VarY-2*torch.sqrt(VarX*VarY+epsilon)
                           #Res=(McX-McY)**2+VarX+VarY
                            Res=Lambda*(abs(McX1-McY1)+abs(McX2-McY2))
                    else:
                        #print(CovX[0,0])
                        if (CovX[0,0] > epsilon) and (CovY[0,0]  > epsilon):
                            #B=torch.trace(CovX)+torch.trace(CovY)-2*torch.sqrt(torch.trace((SqCovX.mm(CovY)).mm(SqCovX)))
                            B=torch.trace(CovX)+torch.trace(CovY)-2*torch.trace(Sq4)
                            Res=Res+(McX1-McY1)**2+(McX2-McY2)**2+B
                            #Res=Res+(McX-McY)**2
                        else :
                            print('Yes1')
                            #Res=Res+(McX-McY)**2+VarX+VarY-2*torch.sqrt(VarX*VarY+epsilon)
                            #Res=Res+(McX-McY)**2+VarX+VarY
                            Res=Res+Lambda*(abs(McX1-McY1)+abs(McX2-McY2))
                else:
                    if (n1==1) and (int(n01+n1)==int(b)):
                        #print('yes2')
                        McX=(1/n1)*X[n01]
                        VarX=torch.zeros(1)  
                    else:
                        #print('yes3')
                        McX=X[n01+n1]
                        VarX=torch.zeros(1)        
                    
                    #print(n01+n1)
                    #print(n02+n2)
                    if (n2!=1):
                        #print('yes1')
                        McY=(1/n2)*torch.sum(Y[n02:n02+n2])
                        VarY=torch.var(Y[n02:n02+n2])
                        
                    elif (n2==1) and (int(n02+n2)==int(b)):
                        #print('yes2')
                        McY=(1/n2)*Y[n02]
                        VarY=torch.zeros(1)  
                        #breakpoint()
                    else:
                        #print('yes3')
                        McY=Y[n02+n2]
                        VarY=torch.zeros(1)   
                        #breakpoint()
                    if k==0:
                        print('Yes2')
                        Res=Lambda*abs(McX-McY)
                    else:
                        print('Yes2')
                        Res=Res+Lambda*abs(McX-McY)

    return Res      





def Wass2DimY(Cc1, Cc2, X,Y, Nm, b):
    Res=0
    for k in range(Nm):
        if k==0:
            n01=0
            n02=0
            
        else:
            n01=n01+int(Cc1[k-1])
            n02=n02+int(Cc2[k-1])
       # if (n02==0):
           # breakpoint()
        n1=int(Cc1[k])
        n2=int(Cc2[k])
        #print(n01+n1)
        #print(n02+n2)
        McX=torch.zeros(1)
        McY=torch.zeros(1)
        VarX=torch.zeros(1)
        VarY=torch.zeros(1)
        epsilon=10**(-16)
        Lambda=100000
        if (n1 != 0) and (n2!= 0):
            if (min(Y[n02:n02+n2,0,0]) <= max(X[n01:n01+n1,0,0])) and (max(Y[n02:n02+n2,0,0]) >= min(X[n01:n01+n1,0,0])):
                if (n1!=1) and (n2!=1):
                    #print('OUI')
                    McX1=(1/n1)*torch.sum(X[n01:n01+n1,0,1])
                    McX2=(1/n1)*torch.sum(X[n01:n01+n1,1,1])

                    #VarX1=torch.var(X[n01:n01+n1,0])
                    #VarX2=torch.var(X[n01:n01+n1,1])

                    McY1=(1/n2)*torch.sum(Y[n02:n02+n2,0,1])
                    McY2=(1/n2)*torch.sum(Y[n02:n02+n2,1,1])

                    #VarY1=torch.var(Y[n02:n02+n2,0])
                    #VarY2=torch.var(Y[n02:n02+n2,1])
                    
                    #breakpoint()
                    CovY=torch.cov(Y[n02:n02+n2,:,1].T)
                    CY=torch.linalg.eig(CovY)
                    
                    CEig=torch.view_as_real(CY[0])
                    S=torch.view_as_real(CY[1])
                    Si=torch.zeros(2,2)
                    Si[:,0]=S[:,0][:,0]
                    Si[:,1]=S[:,1][:,0]
                    #breakpoint()
                    #SqCovX=torch.transpose(Si, 0, 1).mm(torch.diag((CEig[:,0])**(1/2))).mm(Si)
                    SqCovY=Si.mm(torch.diag((CEig[:,0])**(1/2))).mm(torch.transpose(Si, 0, 1))
                    CovX=torch.cov(X[n01:n01+n1,:,1].T)
                    #breakpoint()
                    if k==0:
                        if (CovX[0,0] > epsilon) and (CovY[0,0] > epsilon):
                            B=torch.trace(CovX)+torch.trace(CovY)-2*torch.sqrt(torch.trace((SqCovY.mm(CovX)).mm(SqCovY)))
                            Res=(McX1-McY1)**2+(McX2-McY2)**2+B
                        else :
                            print('Yes1')
                           #Res=(McX-McY)**2+VarX+VarY-2*torch.sqrt(VarX*VarY+epsilon)
                           #Res=(McX-McY)**2+VarX+VarY
                            Res=Lambda*(abs(McX1-McY1)+abs(McX2-McY2))
                    else:
                        #print(CovX[0,0])
                        if (CovX[0,0] > epsilon) and (CovY[0,0]  > epsilon):
                            #print('Yes1')
                            B=torch.trace(CovX)+torch.trace(CovY)-2*torch.sqrt(torch.trace((SqCovY.mm(CovX)).mm(SqCovY)))
                            Res=Res+(McX1-McY1)**2+(McX2-McY2)**2+B
                            #Res=Res+(McX-McY)**2
                        else :
                            print('Yes1')
                            #Res=Res+(McX-McY)**2+VarX+VarY-2*torch.sqrt(VarX*VarY+epsilon)
                            #Res=Res+(McX-McY)**2+VarX+VarY
                            Res=Res+Lambda*(abs(McX1-McY1)+abs(McX2-McY2))
                else:
                    if (n1==1) and (int(n01+n1)==int(b)):
                        #print('yes2')
                        McX=(1/n1)*X[n01]
                        VarX=torch.zeros(1)  
                    else:
                        #print('yes3')
                        McX=X[n01+n1]
                        VarX=torch.zeros(1)        
                    
                    #print(n01+n1)
                    #print(n02+n2)
                    if (n2!=1):
                        #print('yes1')
                        McY=(1/n2)*torch.sum(Y[n02:n02+n2])
                        VarY=torch.var(Y[n02:n02+n2])
                        
                    elif (n2==1) and (int(n02+n2)==int(b)):
                        #print('yes2')
                        McY=(1/n2)*Y[n02]
                        VarY=torch.zeros(1)  
                        #breakpoint()
                    else:
                        #print('yes3')
                        McY=Y[n02+n2]
                        VarY=torch.zeros(1)   
                        #breakpoint()
                    if k==0:
                        print('Yes2')
                        Res=Lambda*abs(McX-McY)
                    else:
                        print('Yes2')
                        Res=Res+Lambda*abs(McX-McY)

    return Res  

def PartitionQuantilesVect(V, Nm, b, d):
    '''
    if ((l%2)==0):
        d=0
        #print('I1')
    else:
        #print('I2')
        d=1
    '''
    Qk=stat.quantiles((V[:,d,0]).detach(),n=Nm)
    Ik=torch.zeros(Nm+1)
    Ik[0]=min(V[:,d,0])
    
    Ik[Nm]=max(V[:,d,0])
    for l in range(0,Nm-1):
        Ik[l+1]=Qk[l]
    
    f=0
    T=torch.zeros(b,2,3);
    i=0
    Cc=torch.zeros(Nm)

    for p in range(Nm):
        s=0
        for l in range(b):
            if (Ik[p] < V[l,d,0]) and (V[l,d,0]<= Ik[p+1] ):
                T[f,0,:-1]=V[l,0,:]
                T[f,1,:-1]=V[l,1,:]
                f=f+1
                s=s+1             
                #print('f=',f,'l=',l,'p=',p)
        if s!= 0:
            T[f-1,-1:]=int(s)
            Cc[i]=int(s)
            i=i+1
        #if Cc[0]==0:
            #breakpoint()
    return (T, Cc)





In [4]:
'''*********************Data set*********************'''

# Data set for the SABR model

alpha = 0.2
beta = 0.8
rho = -0.3
f0 = 100
sigma0 = 0.36
ML = 10000

F_fine, sigma_fine = sabr_trajectory(alpha, beta, rho, f0, sigma0, T, dt1, ML)

StNumML1=np.zeros((ML, 2, ts1.size+1), dtype=np.float32) #Matrix of the input data 
StNumML1[:,0,:]=F_fine
StNumML1[:,1,:]=sigma_fine



In [11]:
'''Networks'''

#Generator Network
NetworkG = Generator(input_neurons = 2, hidden_neurons = 16, output_neurons = 3)

'''Optimizers'''

#Generator optimizer Network
optimizerG = torch.optim.Adam(NetworkG.parameters(), lr=0.001, betas=(0.5, 0.999))



In [ ]:
batche_size=300

Ntrain=ML  #Size of the training set
NetworkG.train()
LossMSE=nn.MSELoss()

'''Partition'''

for epoch in range(100):
    for mini_batches in range(int(Ntrain/batche_size)):
            #mini_batches=0
            Nm=2
            Xtrain=torch.as_tensor(StNumML1[mini_batches*batche_size:(mini_batches+1)*batche_size,:,:])
            lik=0
            Ypi=torch.empty(batche_size,2)
            Ypp=torch.zeros(batche_size,2)
            Ypi[:,:]=Xtrain[:,:,1]
            l=0
            for j in range(1,ts1.size):
                YTorch=Xtrain[:,:,j:j+2]
                #Y, CcY=PartitionQuantilesVect(YTorch, Nm, batche_size, l)
                '''
                if (j!=1):
                    YpiTorch[:,0]=Ypp.detach()
                '''
                ParaG=NetworkG(Ypi)
                #print('ParaG=',ParaG)
                galpha=ParaG[:,0]
                gbeta=ParaG[:,1]
                grho=ParaG[:,2]
                torch.random.seed()
                dW   = torch.normal(0,1,size=(1, 2*batche_size))
                dW21 = dW[0,:batche_size]
                dW31 = grho*dW21+torch.sqrt(1 - grho**2) *dW[0,batche_size:]
                
                Ypp[:,0]=Ypi[:,0]+Ypi[:,1]*(Ypi[:,0]**gbeta)*(dt1**(1/2))*dW21[:]
                Ypp[:,1]=Ypi[:,1]+(galpha)*Ypi[:,1]*(dt1**(1/2))*dW31[:]
                
                Ypi=(Ypp).clone()
                YpT=torch.empty(batche_size,2,2)
                YpT[:,0,:]=torch.cat(( Ypi[:,0].view(-1,1), (Ypp[:,0]).view(-1,1)), dim=1)
                YpT[:,1,:]=torch.cat(( Ypi[:,1].view(-1,1), (Ypp[:,1]).view(-1,1)), dim=1)

                for d in range(2):
                    Y, CcY=PartitionQuantilesVect(YTorch, Nm, batche_size, d)
                    Yp, CcYp=PartitionQuantilesVect(YpT, Nm, batche_size, d)
                    
                    if (j==1):
                            lik=lik+Wass2Dim(CcY, CcYp, Y[:,:,:], (Yp[:,:,:]), Nm, batche_size)
                           # breakpoint()
                    else:
                            lik=lik.clone()+Wass2Dim(CcY, CcYp, Y[:,:,:], (Yp[:,:,:]), Nm, batche_size)  
                            #lik=Wass2Dim(CcY, CcYp, Y[:,:,:], (Yp[:,:,:]), Nm, batche_size)  
                #breakpoint()
                #Generator_loss=torch.sum(Ypp)
                #print('lik=',lik)
                
                #lik=Wass2(CcY, CcYp, Y[:,1], (Yp[:,1]), Nm, batche_size) 
                #lik=LossMSE(Ypp[:,0], YTorch[:,0])
                #breakpoint()
                
                '''
                Generator_loss= lik
                optimizerG.zero_grad()
                Generator_loss.backward(retain_graph=True)
                #print('grad=',Generator_loss.grad)  
                #Genrator network parameters update
                optimizerG.step()
                
                
                #print(Generator_loss)
                #Genrator network optimizer to initialize
                #breakpoint()
                #print(lik)
                print('ParaG=',ParaG)
                print('j=',j)
                #print('lik=',lik)
                print('grad=', Generator_loss)    
                print('epoch=', epoch)
                
                #YpiTorch[:,0]=Ypp.clone() 
                '''
            #print('MeanSigB=',torch.mean(ParaG[:,1]/Ypi[:,0]))
            #print('MeanDriftB=',torch.mean(ParaG[:,0]/Ypi[:,0]))
            Generator_loss = lik
            optimizerG.zero_grad()
            #Genrator network backpropagation
            Generator_loss.backward(retain_graph=True)
            #Genrator network parameters update
            optimizerG.step()
            #print('ParaG=',ParaG)
            print('epoch=', epoch)
            print('Error Generator=', Generator_loss/(Nm*d*N1))
            #print('MeanSigA=',torch.mean(ParaG[:,1]/Ypi[:,0]))
            #print('MeanDriftA=',torch.mean(ParaG[:,0]/Ypi[:,0]))
            print('MeanBeta=',torch.mean(ParaG[:,1]))
            print('MeanAlpha=',torch.mean(ParaG[:,0]))
            print('grho=',torch.mean(ParaG[:,2]))
            
            

            
            #breakpoint()

